# Introduction

This is a continuation of the `dice_sample` and `dice_posterior` assignments.
In these assignments, we have a bag containing two types of dice with different
probabilities of rolling each number (also referred to as a "face" of the die).
Someone selects a die from the bag at random, rolls it a fixed number of times,
reports the outcomes, returns it to the bag, and repeats the process. We refer
each selection of a new die as a "draw". Here, you will write code to run the EM
(Expectation Maximization) algorithm to estimate the parameters of the system --
the probability of drawing each die type and the conditional probability of each
face given the die type.

This notebook provides a brief overview of the assignment but you'll have to
read the code for detailed instructions.  

All of the graded tests are written such that only two dice are in the bag.
Feel free write the code such that it only functions on two dice or to write 
more general code which can operate on bags of any composition.

## Die and BagOfDice classes
For this assignment, you will use our implementation of these two classes. These 
lines at the top of the assignment file make those two classes availble from our 
cse587Autils package:
```python
from cse587Autils.DiceObjects.Die import Die
from cse587Autils.DiceObjects.BagOfDice import BagOfDice
```
These are more full featured implementations than the code we provided with
the `dice-posterior` assignment.


### Die class
Instantiate with a list of the probabilities of rolling each face on the die.
The list can be of any length but the probabilities must sum to one.
```python
biased_die = Die(face_probs=[0.1, 0.2, 0.7])
print(biased_die) # prints Die([0.1, 0.2, 0.7])
```
You can get the number of faces with the <len> function
```python
len(biased_die) # returns 3
```
Previously, we used `num_faces` for that. You can also index into the die object
directly to get face probabilities:

```python
biased_die[1] # returns 0.2. Note zero-based indexing.
```

### BagOfDice class
Instantiate with two arguments. The first a list of the probabilies of drawing
each die type, which must sum to one. The second is a list of Die objects 
representing the unique die types in the bag (each type is listed only once).

In [1]:
from cse587Autils.DiceObjects.BagOfDice import BagOfDice, Die
fair_die = Die([1/3]*3) # In python, this is equivalent to Die[1/3, 1/3, 1/3]
biased_die = Die(face_probs=[0.1, 0.2, 0.7])

bag = BagOfDice([0.25, 0.75],[fair_die, biased_die])
print(bag)

BagOfDice(die_priors=[0.25, 0.75], dice=[Die([0.3333, 0.3333, 0.3333]), Die([0.1, 0.2, 0.7])])


Indexing into the bag returns tuple of the corresponding die's probability of
being drawn and the die object itself. You can also iterate through the dice.

In [2]:
bag[1] # Returns (0.75, Die([0.1, 0.2, 0.7]))
for prior_die_pair in bag:
    die_prior, die = prior_die_pair
    print(f'die prior: {die_prior}\ndie object: {die}')

die prior: 0.25
die object: Die([0.3333, 0.3333, 0.3333])
die prior: 0.75
die object: Die([0.1, 0.2, 0.7])


BagOfDice supports the following operations:

In [3]:
# len() returns the number of dice in the bag
print(f'len() returns the number of dice in the bag: {len(bag)}')

# there are getters (and setters) for the die_priors and dice
print(f'accessing the die priors: {bag.die_priors}')
print(f'accessing the Die objects: {bag.dice}')

len() returns the number of dice in the bag: 2
accessing the die priors: [0.25, 0.75]
accessing the Die objects: [Die([0.3333, 0.3333, 0.3333]), Die([0.1, 0.2, 0.7])]


We can use the BagOfDice to generate sample data. For example, to 
produce a sample by drawing a die, with replacement, from the bag 5 times 
and rolling each die drawn 20 times, we can use the draw() method. `draw()`
returns a numpy array containg the number of times each face was rolled on a 
given draw. The array length is the maximum number of faces among all the dice 
in the bag. Faces that were not rolled (either by chance or because they don't 
exist on the die) are represented by 0.

In [8]:
sample_data = [bag.draw(20) for _ in range(5)]
print(sample_data)

[array([7, 9, 4]), array([8, 5, 7]), array([ 0,  6, 14]), array([ 3,  2, 15]), array([ 0,  5, 15])]


In your EM implementation, you will use a BagOfDice object to store the current 
estimates of the parameters. On each call to `m_step` you will create a new bag
with updated parameter estimations.

Your EM algorithm must stop when it converges -- that is, when the change in 
parameter estimates from one iteration to the next becomes very small. To
characterize the change in parameters, we will use the sum of the absolute values
of the differences between between the parameters, include die priors and face
probabilities. We have implemented this measure for you. To get it, simply use
the subtraction operator on two BagOfDice objects, as shown below. This will 
compare the first die of one bag to the first die of the other, etc. The order
of dice is assumed to be the same, but this is not a problem for EM iterations --
your estimates for each die will always be in the same order.

In [9]:
bag1 = BagOfDice([0.5, 0.5], 
                 [Die([0.9, 0.1, 0.0]), Die([0.1, 0.1, 0.8])])
bag2 = BagOfDice([0.6, 0.4], 
                 [Die([0.9, 0.1, 0.0]), Die([0.1, 0.2, 0.7])])
print(f'bag difference: {bag1 - bag2}')

bag difference: 0.4


## Input and Output

The top level function is called diceEM:

```python
def diceEM(experiment_data: List[NDArray[np.int_]],
           bag_of_dice: BagOfDice,
           accuracy: float,
           max_iterations: int = int(1e4)) -> [int, BagOfDice]:
```

where,

- `experiment_data`  is a list of draws, each of which gives the results of
drawing a die from the bag and rolling it n times and then aggregating the
number of times each face appears. It is a list of numpy arrays, where the number 
of arrays is equal to the number of `draws` that is performed, and the sum of
each individual array is the total number of times the die was rolled on that
draw.

- `max_iterations` similarly sets a threshold at the number of iterations the EM
algorithm can run -- it is a good idea to set this to avoid endless, or
needlessly long, loops. But, you may need to adjust this value depending on the
accuracy. Default is 10000.

The number of faces, trials per draw, etc., can all be calculated from the input
data and so will not be provided. 

`diceEM()` implements the outer loop of the EM algorithm. It calls `e_step()`
and `m_step()` on each iteration and updates its estimate of the parameters
which generated the `experiment_data`. This updated estimate is used in the
sebsequent iteration, and so on until the stop conditions are met.

The return value of `diceEM()` is the final estimate of the parameters in the 
form of a `BagOfDice` object.

Your `e_step()` code needs to calculate posteriors. You are welcome to use your 
dice_posterior code. If you did *not* have functioning `dice_posterior()` code, 
or you question its accuracy after pasting it in to this assignment, the TAs can
provide correct dice_posterior code to you.

Outlines of the code are provided in the file [assignment.py](assignment.py).
Read the comments, too. You need to fill in key parts of the algorithms. Feel 
free to paste your dice_posterior code in.  

**Important**: your code will be graded against the
[test_assignment.py](test_assignment.py) code exactly as it is when you received
this assignment. You are welcome to change it in your project, but those changes
will *not* be reflected when your code is graded. So, we suggest you *do not*
change the tests. Otherwise, you will potentially be surprised if your code
fails during the automated grading.

## EM Initialization Tips

To initialize the parameter estimates, do not make all the possibilities
equally likely. If you do, the algorithm may get stuck and take longer to
converge. However, do not make them
too far from equally likely, either, to avoid strongly biasing the final result
by the initial values. Since there are only two die types, I suggest
initializing their probabilities to 0.45 and 0.55. For the probabilities of the
faces of each die, I took a random real between 1/n and 2/n, where n is the
number of faces. Then I normalized them so they would all add up to one using the call

```
list_of_numbers / sum(list_of_numbers)
```

## Experiments and questions

### Number of iterations needed for convergence

Given a very large dataset, how many iterations does it take to converge to within the required accuracy of 10^-4? How does that change if you tighten the convergence criterion to 10^-6? How about 10^-8. Please make a comment on the general pattern you observe.

In [28]:
from importlib import reload
import diceEM.assignment as assign
reload(assign) # This ensures code is reloaded each time
from diceEM.assignment import diceEM, generate_sample

experiment_result = generate_sample([4,6], 
                                    [[0.3, 0.2, 0.2, 0.3], 
                                     [0, 0.2, 0.2, 0.6]], 
                                     4000, 
                                     30)

init_bag = BagOfDice([0.75, 0.25], 
                     [Die([0.1, 0.3, 0.5, 0.1]), 
                      Die([0.1, 0.3, 0.4, 0.2])])
diceEM(experiment_result, init_bag, 10e-4, 100)

(6,
 BagOfDice(die_priors=[0.40896643154383977, 0.5910335684561603], dice=[Die([0.2964, 0.1988, 0.202, 0.3028]), Die([0.0, 0.1994, 0.1987, 0.6018])]))

Answer: 10^-4 takes 6 iterations.Tightening the convergence criteria increases the iterations, but not by much. 10^-6 takes 9 iterations 10^-8 takes 12 iterations.

How much effect does tightening the accuracy requirement by 2 or 4 orders of magnitude have on how well the algorithm identifies the correct parameters?

Answer: Tightening the accuracy in this case has a negligible effect on how well the algorithm identifies the correct parameters.

Now let's change the parameters of the sample generated as shown below. 

How does this affect the number of iterations required to make the accuracy goal? Why do you think that is?

In [25]:
from importlib import reload
import diceEM.assignment as assign
reload(assign) # This ensures code is reloaded each time
from diceEM.assignment import diceEM, generate_sample

experiment_result = generate_sample([4, 6], [[0.3, 0.2, 0.3, 0.2], 
                                             [0.1, 0.2, 0.2, 0.5]], 
                                             4000, 
                                             30)

init_bag = BagOfDice([0.75, 0.25], [Die([0.1, 0.3, 0.5, 0.1]), 
                                    Die([0.1, 0.3, 0.4, 0.2])])

diceEM(experiment_result, init_bag, 10e-4, 100)

(9,
 BagOfDice(die_priors=[0.40416219790208086, 0.5958378020979191], dice=[Die([0.2973, 0.1988, 0.304, 0.2]), Die([0.1004, 0.1993, 0.1998, 0.5005])]))

Answer: This increases the number of iterations to 9, this likely occurs because the initial parameters present more variation than the converging result of the EM algorithm.

### Sample size

1. Consider the following experiment.

In [29]:
from importlib import reload
import diceEM.assignment as assign
reload(assign) # This ensures code is reloaded each time
from diceEM.assignment import diceEM, generate_sample

face_probs = [[0.3, 0.3, 0.2, 0.2],
              [0.2, 0.2, 0.2, 0.4]]

experiment_result = generate_sample([4, 6], face_probs, 6000, 30)
init_bag = BagOfDice([0.75, 0.25], [Die([0.1, 0.3, 0.5, 0.1]), 
                                    Die([0.1, 0.3, 0.4, 0.2])])

inference_result = diceEM(experiment_result, init_bag, 10e-4, 100)
print(inference_result[1])

BagOfDice(die_priors=[0.40254223047228077, 0.5974577695277191], dice=[Die([0.2986, 0.3027, 0.2015, 0.1972]), Die([0.1988, 0.1986, 0.2011, 0.4015])])


- How many die rolls are in this sample? 

Answer: There are 6000*30 = 180000 die rolls in this sample

- Try reducing the number of draws/trials while keeping the number of rolls constant. How many draws do you need with 30 rolls each to get the right answer consistently, to within two decimal places? 

In [33]:
for data_seed in range(10):
    data = generate_sample([4, 6], face_probs, 10000, 30, seed=data_seed)
    print(diceEM(data, init_bag, 10e-4, 100))

(23, BagOfDice(die_priors=[0.4080597956711333, 0.5919402043288667], dice=[Die([0.2993, 0.2996, 0.1996, 0.2016]), Die([0.201, 0.198, 0.2, 0.401])]))
(23, BagOfDice(die_priors=[0.3848665306318865, 0.6151334693681134], dice=[Die([0.3022, 0.3014, 0.1989, 0.1974]), Die([0.2024, 0.2003, 0.1995, 0.3979])]))
(24, BagOfDice(die_priors=[0.4093826127608863, 0.5906173872391137], dice=[Die([0.3037, 0.2974, 0.1991, 0.1999]), Die([0.2017, 0.2011, 0.1978, 0.3995])]))
(24, BagOfDice(die_priors=[0.41013414561484657, 0.5898658543851534], dice=[Die([0.2982, 0.3007, 0.1997, 0.2013]), Die([0.2008, 0.1998, 0.1986, 0.4007])]))
(24, BagOfDice(die_priors=[0.40296704712556747, 0.5970329528744325], dice=[Die([0.2996, 0.3015, 0.1981, 0.2008]), Die([0.2021, 0.2001, 0.1986, 0.3991])]))
(23, BagOfDice(die_priors=[0.39550275311923744, 0.6044972468807626], dice=[Die([0.2968, 0.3018, 0.2014, 0.2]), Die([0.1997, 0.2017, 0.1986, 0.3999])]))
(24, BagOfDice(die_priors=[0.3907363396356911, 0.6092636603643089], dice=[Die([0.3

Answer: Approximately 10,000 draws were needed to get the right answer consistently, even though there were some trials that were not within 2 decimal places.

- If you don't have quite enough draws/trials, which parameters tend to be off?

The priors were more off than the face probs.

- Now try changing the number of rolls per trial to 50. How many trials do you 
need to consistently get the right  answer consistently, to within two decimal places? 

In [37]:
for data_seed in range(10):
    data = generate_sample([4, 6], face_probs, 3000, 50, seed=data_seed)
    print(diceEM(data, init_bag, 10e-8, 100))

(31, BagOfDice(die_priors=[0.40090942622617987, 0.59909057377382], dice=[Die([0.3011, 0.3005, 0.1973, 0.2011]), Die([0.2006, 0.1974, 0.2002, 0.4019])]))
(30, BagOfDice(die_priors=[0.404264188606803, 0.5957358113931969], dice=[Die([0.2997, 0.2984, 0.1999, 0.2021]), Die([0.2021, 0.1973, 0.2003, 0.4003])]))
(31, BagOfDice(die_priors=[0.40640691214391306, 0.5935930878560869], dice=[Die([0.3034, 0.2984, 0.1976, 0.2006]), Die([0.2013, 0.2022, 0.2, 0.3965])]))
(28, BagOfDice(die_priors=[0.41093409779241513, 0.5890659022075847], dice=[Die([0.3009, 0.2981, 0.1997, 0.2012]), Die([0.1992, 0.1989, 0.1987, 0.4033])]))
(29, BagOfDice(die_priors=[0.40147701691949383, 0.5985229830805062], dice=[Die([0.3004, 0.2995, 0.1992, 0.201]), Die([0.2022, 0.199, 0.2, 0.3988])]))
(27, BagOfDice(die_priors=[0.39351147467621495, 0.6064885253237852], dice=[Die([0.2978, 0.301, 0.2013, 0.2]), Die([0.1996, 0.198, 0.2004, 0.402])]))
(30, BagOfDice(die_priors=[0.3901057517417996, 0.6098942482582004], dice=[Die([0.3041, 0

Answer: Now, approximately 3000-4000 draws are needed to get the right answer consistently, even though there were some trials not within 2 decimal places.

- Does tightening the convergence criterion make the results substantially more accurate? 

Answer: I would not say substantiallym, but they seem to help make the results slightly more accurate.